<a href="https://colab.research.google.com/github/german1728/Chimera/blob/master/Tutorial4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 4
# Ajuste del modelo: regresión lineal múltiple y regresión polinomial

**Content creators**: Pierre-Étienne Fiquet, Anqi Wu, Alex Hyafil with help from Byron Galbraith, Ella Batty

**Content reviewers**: Lina Teichmann, Saeed Salehi, Patrick Mineault, Michael Waskom

**Content traduction**: Israel Chaparro

Source: Neuromatch Academy

---
#Objetivos del Tutorial

Este es el Tutorial 4 de una serie sobre cómo ajustar modelos a datos. Comenzamos con regresión lineal simple, usando optimización de mínimos cuadrados (Tutorial 1) y Estimación de máxima verosimilitud (Tutorial 2). Usaremos bootstrapping para construir intervalos de confianza alrededor de los parámetros del modelo lineal inferido (Tutorial 3). Terminaremos nuestra exploración de modelos de regresión generalizando a la regresión lineal múltiple y la regresión polinomial (Tutorial 4). Terminamos aprendiendo a elegir entre estos distintos modelos. Analizamos la compensación de sesgo-varianza (Tutorial 5) y la Validación cruzada para la selección del modelo (Tutorial 6).

En este tutorial, generalizaremos el modelo de regresión para incorporar múltiples características.
- Aprenda a estructurar las entradas para la regresión utilizando la 'Matriz de diseño'
- Generalizar el MSE para múltiples características usando el estimador de mínimos cuadrados ordinario
- Visualice datos y ajuste del modelo en múltiples dimensiones
- Ajustar modelos de regresión polinomial de diferente complejidad
- Trazar y evaluar los ajustes de regresión polinomial.

---
# Configuración

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#@title Configuración de Figuras
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

In [3]:
#@title Funciones de Ayuda

def plot_fitted_polynomials(x, y, theta_hat):
  """ Plot polynomials of different orders

  Args:
    x (ndarray): input vector of shape (n_samples)
    y (ndarray): vector of measurements of shape (n_samples)
    theta_hat (dict): polynomial regression weights for different orders
  """

  x_grid = np.linspace(x.min() - .5, x.max() + .5)

  plt.figure()

  for order in range(0, max_order + 1):
    X_design = make_design_matrix(x_grid, order)
    plt.plot(x_grid, X_design @ theta_hat[order]);

  plt.ylabel('y')
  plt.xlabel('x')
  plt.plot(x, y, 'C0.');
  plt.legend([f'order {o}' for o in range(max_order + 1)], loc=1)
  plt.title('polynomial fits')
  plt.show()

---
# Sección 1: Regresión lineal múltiple


Ahora que hemos considerado el caso univariado y cómo producir intervalos de confianza para nuestro estimador, pasamos al caso de regresión lineal general, donde podemos tener más de un regresor o característica en nuestra entrada.

Recuerde que nuestro modelo lineal univariado original se dio como

\begin{align}
y = \theta x + \epsilon
\end{align}

donde $ \theta $ es la pendiente y $ \epsilon $ algo de ruido. Podemos extender esto fácilmente al escenario multivariado agregando otro parámetro para cada característica adicional

\begin{align}
y = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + ... +\theta_d x_d + \epsilon
\end{align}

donde $ \theta_0 $ es la intersección y $ d $ es el número de características (también es la dimensionalidad de nuestra entrada).

Podemos condensar esto sucintamente usando notación vectorial para un solo punto de datos

\begin{align}
y_i = \boldsymbol{\theta}^{\top}\mathbf{x}_i + \epsilon
\end{align}

y completamente en forma de matriz

\begin{align}
\mathbf{y} = \mathbf{X}\boldsymbol{\theta} + \mathbf{\epsilon}
\end{align}

donde $ \mathbf {y} $ es un vector de medidas, $ \mathbf {X} $ es una matriz que contiene los valores de las características (columnas) para cada muestra de entrada (filas), y $ \boldsymbol{\theta} $ es nuestro vector de parámetro.

Para este tutorial nos centraremos en el caso bidimensional ($ d = 2 $), que nos permite visualizar fácilmente nuestros resultados. Como ejemplo, piense en una situación en la que un científico registra la respuesta en picos de una célula ganglionar de la retina a patrones de señales de luz que varían en contraste y orientación. Luego, los valores de contraste y orientación se pueden usar como características / regresores para predecir la respuesta de las células.

En este caso, nuestro modelo se puede escribir como:

\begin{align}
y = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \epsilon
\end{align}

o en forma de matriz donde

\begin{align}
\mathbf{X} = 
\begin{bmatrix}
1 & x_{1,1} & x_{1,2} \\
1 & x_{2,1} & x_{2,2} \\
\vdots & \vdots & \vdots \\
1 & x_{n,1} & x_{n,2}
\end{bmatrix}, 
\boldsymbol{\theta} =
\begin{bmatrix}
\theta_0 \\
\theta_1 \\
\theta_2 \\
\end{bmatrix}
\end{align}

Para nuestro conjunto de datos de exploración real, estableceremos $ \boldsymbol{\theta} = [0, -2, -3] $ y sacaremos $ N = 40 $ muestras ruidosas de $ x \in [-2,2) $. Tenga en cuenta que establecer el valor de $ \theta_0 = 0 $ efectivamente ignora el término de compensación.

In [ ]:
#@title
#@markdown Ejecute esta celda para simular algunos datos

# Set random seed for reproducibility
np.random.seed(1234)

# Set parameters
theta = [0, -2, -3]
n_samples = 40

# Draw x and calculate y
n_regressors = len(theta)
x0 = np.ones((n_samples, 1))
x1 = np.random.uniform(-2, 2, (n_samples, 1))
x2 = np.random.uniform(-2, 2, (n_samples, 1))
X = np.hstack((x0, x1, x2))
noise = np.random.randn(n_samples)
y = X @ theta + noise


ax = plt.subplot(projection='3d')
ax.plot(X[:,1], X[:,2], y, '.')

ax.set(
    xlabel='$x_1$',
    ylabel='$x_2$',
    zlabel='y'
)
plt.tight_layout()

Ahora que tenemos nuestro conjunto de datos, queremos encontrar un vector óptimo de parámetros $ \boldsymbol {\hat\theta} $. Recuerde nuestra solución analítica para minimizar MSE para un solo regresor:

\begin{align}
\hat\theta = \frac{\sum_{i=1}^N x_i y_i}{\sum_{i=1}^N x_i^2}.
\end{align}

Lo mismo es cierto para el caso del regresor múltiple, solo que ahora se expresa en forma de matriz

\begin{align}
\boldsymbol{\hat\theta} = (\mathbf{X}^\top\mathbf{X})^{-1}\mathbf{X}^\top\mathbf{y}.
\end{align}

### Ejercicio 1: Estimador ordinario de mínimos cuadrados

En este ejercicio, implementará el enfoque MCO para estimar $ \boldsymbol{\hat \theta} $ a partir de la matriz de diseño $ \mathbf {X} $ y el vector de medición $ \mathbf {y} $. Puede utilizar el símbolo `@` para la multiplicación de matrices, `.T` para la transposición y` np.linalg.inv` para la inversión de matrices.


In [ ]:
def ordinary_least_squares(X, y):
  """Ordinary least squares estimator for linear regression.

  Args:
    x (ndarray): design matrix of shape (n_samples, n_regressors)
    y (ndarray): vector of measurements of shape (n_samples)

  Returns:
    ndarray: estimated parameter values of shape (n_regressors)
  """

  # Computar theta_hat usando OLS
  theta_hat = ...

  return theta_hat


# Descomente a continuación para probar su función
# theta_hat = ordinary_least_squares(X, y)
# print(theta_hat)

[*Haga clic para obtener una solución*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W1D3_ModelFitting/solutions/W1D3_Tutorial4_Solution_2b5abc38.py)



Después de completar esta función, debería ver que $\hat{\theta}$ = [ 0.13861386, -2.09395731, -3.16370742]

Ahora que tenemos nuestro $ \mathbf {\hat \theta} $, podemos obtener $ \mathbf {\hat y} $ y por lo tanto nuestro error cuadrático medio.

In [ ]:
# Calcular datos predichos
theta_hat = ordinary_least_squares(X, y)
y_hat = X @ theta_hat

# Computar MSE
print(f"MSE = {np.mean((y - y_hat)**2):.2f}")

Finalmente, el siguiente código trazará una visualización geométrica de los puntos de datos (azul) y el plano ajustado.

In [ ]:
#@title
#@markdown Ejecute esta celda para visualizar datos y plano predicho

theta_hat = ordinary_least_squares(X, y)
xx, yy = np.mgrid[-2:2:50j, -2:2:50j]
y_hat_grid = np.array([xx.flatten(), yy.flatten()]).T @ theta_hat[1:]
y_hat_grid = y_hat_grid.reshape((50, 50))

ax = plt.subplot(projection='3d')
ax.plot(X[:, 1], X[:, 2], y, '.')
ax.plot_surface(xx, yy, y_hat_grid, linewidth=0, alpha=0.5, color='C1',
                cmap=plt.get_cmap('coolwarm'))

for i in range(len(X)):
  ax.plot((X[i, 1], X[i, 1]),
          (X[i, 2], X[i, 2]),
          (y[i], y_hat[i]),
          'g-', alpha=.5)

ax.set(
    xlabel='$x_1$',
    ylabel='$x_2$',
    zlabel='y'
)
plt.tight_layout()

---
# Sección 2: Regresión polinomial

Hasta ahora, aprendió a predecir resultados a partir de entradas ajustando un modelo de regresión lineal. Ahora podemos modelar todo tipo de relaciones, ¡incluso en neurociencia!

Un problema potencial con este enfoque es la simplicidad del modelo. La regresión lineal, como su nombre lo indica, solo puede capturar una relación lineal entre las entradas y las salidas. Dicho de otra manera, las salidas previstas son solo una suma ponderada de las entradas. ¿Qué pasa si ocurren cálculos más complicados? Afortunadamente, existen muchos modelos más complejos (y encontrará muchos más en las próximas 3 semanas). Un modelo que aún es muy simple de ajustar y comprender, pero que captura relaciones más complejas, es la **regresión polinomial**, una extensión de la regresión lineal.

Dado que la regresión polinomial es una extensión de la regresión lineal, ¡todo lo que aprendiste hasta ahora te resultará útil! El objetivo es el mismo: queremos predecir la variable dependiente $ y_ {n} $ dados los valores de entrada $ x_ {n} $. El cambio clave es el tipo de relación entre entradas y salidas que el modelo puede capturar.

Los modelos de regresión lineal predicen las salidas como una suma ponderada de las entradas:

$$y_{n}= \theta_0 + \theta x_{n} + \epsilon_{n}$$

Con la regresión polinomial, modelamos las salidas como una ecuación polinomial basada en las entradas. Por ejemplo, podemos modelar las salidas como:

$$y_{n}= \theta_0 + \theta_1 x_{n} + \theta_2 x_{n}^2 + \theta_3 x_{n}^3 + \epsilon_{n}$$

Podemos cambiar la complejidad de un polinomio cambiando el orden del polinomio. El orden de un polinomio se refiere a la potencia más alta del polinomio. La ecuación anterior es un polinomio de tercer orden porque el valor más alto de x se eleva a 3. Podríamos agregar otro término ($ + \theta_4 x_{n} ^ 4 $) para modelar un polinomio de orden 4 y así sucesivamente.



Primero, simularemos algunos datos para practicar el ajuste de modelos de regresión polinomial. Generaremos entradas aleatorias $ x $ y luego calcularemos y de acuerdo con $ y = x ^ 2 - x - 2 $, con algo de ruido adicional tanto en la entrada como en la salida para hacer que el ejercicio de ajuste del modelo se acerque más a una situación de la vida real.

In [ ]:
#@title
#@markdown Ejecute esta celda para simular algunos datos

# setting a fixed seed to our random number generator ensures we will always
# get the same psuedorandom number sequence
np.random.seed(121)
n_samples = 30
x = np.random.uniform(-2, 2.5, n_samples)  # inputs uniformly sampled from [-2, 2.5)
y =  x**2 - x - 2   # computing the outputs

output_noise = 1/8 * np.random.randn(n_samples)
y += output_noise  # adding some output noise

input_noise = 1/2 * np.random.randn(n_samples)
x += input_noise  # adding some input noise

fig, ax = plt.subplots()
ax.scatter(x, y)  # produces a scatter plot
ax.set(xlabel='x', ylabel='y');

## Sección 2.1: Matriz de diseño para regresión polinomial

Ahora que tenemos la idea básica de la regresión polinomial y algunos datos ruidosos, ¡comencemos! La diferencia clave entre ajustar un modelo de regresión lineal y un modelo de regresión polinomial radica en cómo estructuramos las variables de entrada.

Para la regresión lineal, usamos $ X = x $ como datos de entrada. Para agregar un sesgo constante (una intersección con el eje y en una gráfica 2-D), usamos $ X = \big[\boldsymbol 1, x \big] $, donde $ \boldsymbol 1 $ es una columna de unos. Al ajustar, aprendemos un peso para cada columna de esta matriz. Entonces aprendemos un peso que se multiplica con la columna 1; en este caso, esa columna es todos unos, por lo que obtenemos el parámetro de sesgo ($ + \theta_0 $). También aprendemos un peso para cada columna, o cada característica de x, como aprendimos en la Sección 1.

Esta matriz $ X $ que usamos para nuestras entradas se conoce como **matriz de diseño**. Queremos crear nuestra matriz de diseño para que aprendamos los pesos de $ x ^ 2, x ^ 3, $, etc. Por lo tanto, queremos construir nuestra matriz de diseño $ X $ para la regresión polinomial de orden $ k $ como:

$$ X = \big [\boldsymbol 1, x ^ 1, x ^ 2, \ldots, x ^ k \big], $$

donde $ \boldsymbol {1} ​​$ es el vector de la misma longitud que $ x $ que consta de todos unos, y $ x ^ p $ es el vector o matriz $ x $ con todos los elementos elevados a la potencia $ p $. Tenga en cuenta que $ \boldsymbol {1} ​​= x ^ 0 $ y $ x ^ 1 = x $

### Ejercicio 2: Matriz de diseño de estructura

Cree una función (`make_design_matrix`) que estructura la matriz de diseño dados los datos de entrada y el orden del polinomio que desea ajustar. Imprimiremos parte de esta matriz de diseño para nuestros datos y ordenaremos 5.

In [ ]:
def make_design_matrix(x, order):
  """Create the design matrix of inputs for use in polynomial regression

  Args:
    x (ndarray): input vector of shape (n_samples)
    order (scalar): polynomial regression order

  Returns:
    ndarray: design matrix for polynomial regression of shape (samples, order+1)
  """
  # Transformamos a la dimensión (n x 1) para que funcione
  if x.ndim == 1:
    x = x[:, None]

  #si x tiene más de una característica, no queremos múltiples columnas de unos, así que asignamos x^0 aquí
  design_matrix = np.ones((x.shape[0], 1))

  # Recorre el resto de grados y apila columnas (pista: np.hstack)
  for degree in range(1, order + 1):
      design_matrix = ...

  return design_matrix


# Descomenta para probar tu función
order = 5
# X_design = make_design_matrix(x, order)
# print(X_design[0:2, 0:2])

[*Haga clic para obtener una solución*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W1D3_ModelFitting/solutions/W1D3_Tutorial4_Solution_f8576b48.py)



Debería ver que la sección impresa de esta matriz de diseño es `[[ 1.         -1.51194917]
 [ 1.         -0.35259945]]`

## Sección 2.2: Ajuste de modelos de regresión polinomial

Ahora que tenemos las entradas estructuradas correctamente en nuestra matriz de diseño, ajustar una regresión polinomial es lo mismo que ajustar un modelo de regresión lineal. Toda la estructura polinomial que necesitamos aprender está contenida en cómo se estructuran las entradas en la matriz de diseño. Podemos usar la misma solución de mínimos cuadrados que calculamos en ejercicios anteriores.

### Ejercicio 3: Ajuste de modelos de regresión polinomial con diferentes órdenes

Aquí, ajustaremos modelos de regresión polinomial para encontrar los coeficientes de regresión ($ \theta_0, \theta_1, \theta_2, $ ...) resolviendo el problema de mínimos cuadrados. Cree una función `solve_poly_reg` que recorra polinomios de diferentes órdenes (hasta ` max_order`), se ajuste a ese modelo y guarde los pesos para cada uno. Puede invocar la función `common_least_squares`.

Luego, inspeccionaremos cualitativamente la calidad de nuestros ajustes para cada orden trazando los polinomios ajustados sobre los datos. Para ver curvas suaves, evaluamos los polinomios ajustados en una cuadrícula de valores $ x $ (que varían entre la mayor y la menor de las entradas presentes en el conjunto de datos).

In [ ]:
def solve_poly_reg(x, y, max_order):
  """Fit a polynomial regression model for each order 0 through max_order.

  Args:
    x (ndarray): input vector of shape (n_samples)
    y (ndarray): vector of measurements of shape (n_samples)
    max_order (scalar): max order for polynomial fits

  Returns:
    dict: fitted weights for each polynomial model (dict key is order)
  """

  # Create a dictionary with polynomial order as keys,
  # and np array of theta_hat (weights) as the values
  theta_hats = {}

  # Loop over polynomial orders from 0 through max_order
  for order in range(max_order + 1):

    # Crear la matriz de diseño
    X_design = ...

    # Ajustar modelo polinomial
    this_theta = ...

    theta_hats[order] = this_theta

  return theta_hats


# Descomenta para probar tu función
max_order = 5
# theta_hats = solve_poly_reg(x, y, max_order)
# plot_fitted_polynomials(x, y, theta_hats)

[*Haga clic para obtener una solución*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W1D3_ModelFitting/solutions/W1D3_Tutorial4_Solution_9b1eebd9.py)

*Salida de ejemplo:*

<img alt='Solution hint' align='left' width=560 height=416 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/W1D3_ModelFitting/static/W1D3_Tutorial4_Solution_9b1eebd9_0.png>



## Sección 2.4: Evaluación de la calidad del ajuste

Al igual que con la regresión lineal, podemos calcular el error cuadrático medio (MSE) para tener una idea de qué tan bien se ajusta el modelo a los datos.

Calculamos MSE como:

$$ MSE = \frac 1 N ||y - \hat y||^2 = \frac 1 N \sum_{i=1}^N (y_i - \hat y_i)^2 $$

donde los valores predichos para cada modelo vienen dados por $ \hat y = X \hat \theta $.

* ¿Qué modelo (es decir, qué orden de polinomio) cree que tendrá el mejor MSE? *

### Ejercicio 4: Calcule MSE y compare modelos

Compararemos el MSE para diferentes órdenes de polinomios con un diagrama de barras.

In [ ]:
mse_list = []
order_list = list(range(max_order + 1))

for order in order_list:

  X_design = make_design_matrix(x, order)

  # Obtenga la predicción para el modelo de regresión polinomial de este orden
  y_hat = ...

  # Calcule los residuos
  residuals = ...

  # Calcule el MSE
  mse = ...

  mse_list.append(mse)

fig, ax = plt.subplots()

# Descomentar una vez que el ejercicio anterior esté completo
# ax.bar(order_list, mse_list)

ax.set(title='Comparing Polynomial Fits', xlabel='Polynomial order', ylabel='MSE')

[*Haga clic para obtener una solución*](https://github.com/NeuromatchAcademy/course-content/tree/master//tutorials/W1D3_ModelFitting/solutions/W1D3_Tutorial4_Solution_ccea5dd7.py)

*Salida de Ejemplo:*

<img alt='Solution hint' align='left' width=558 height=414 src=https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/tutorials/W1D3_ModelFitting/static/W1D3_Tutorial4_Solution_ccea5dd7_0.png>



---
# Resumen

* La regresión lineal se generaliza naturalmente a múltiples dimensiones.
* El álgebra lineal nos brinda las herramientas matemáticas para razonar y resolver tales problemas más allá del caso bidimensional

* Para cambiar de un modelo de regresión lineal a un modelo de regresión polinomial, solo tenemos que cambiar cómo se estructuran los datos de entrada

* Podemos elegir la complejidad del modelo cambiando el orden del ajuste del modelo polinomial

* Los modelos polinomiales de orden superior tienden a tener un MSE más bajo en los datos con los que se ajustan

** Nota **: En la práctica, los problemas multidimensionales de mínimos cuadrados se pueden resolver de manera muy eficiente (gracias a rutinas numéricas como LAPACK).

